# 1. Work on audio
## 1.1. Convert to wav

In [17]:
from pydub import AudioSegment

def mp3_to_wav(mp3_file, wav_file):
    # Load MP3 file
    audio = AudioSegment.from_mp3(mp3_file)
    
    # Export as WAV file
    audio.export(wav_file, format="wav")

# Example usage
mp3_file = "/home/sosa/BI/small.mp3"
wav_file = "/home/sosa/BI/small.wav"
mp3_to_wav(mp3_file, wav_file)

## 1.2. Noice reduction with DeepFilterNet

In [18]:
from df.enhance import enhance, init_df, load_audio, save_audio
model, df_state, _ = init_df()  # Load default model
audio, _ = load_audio("/home/sosa/BI/Audio recording 2017-10-31 15-31-59.wav", sr=df_state.sr())
enhanced_audio = enhance(model, df_state, audio)
save_audio("enhanced.wav", enhanced_audio, df_state.sr())

2024-03-22 11:12:22 | INFO     | DF | Loading model settings of DeepFilterNet3
2024-03-22 11:12:22 | INFO     | DF | Using DeepFilterNet3 model at /home/sosa/.cache/DeepFilterNet/DeepFilterNet3
2024-03-22 11:12:22 | INFO     | DF | Initializing model `deepfilternet3`


2024-03-22 11:12:27 | INFO     | DF | Found checkpoint /home/sosa/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2024-03-22 11:12:27 | INFO     | DF | Running on device cuda:0
2024-03-22 11:12:27 | INFO     | DF | Model loaded


/home/sosa/.local/lib/python3.10/site-packages/df/io.py:106: UserWarning: "sinc_interpolation" resampling method name is being deprecated and replaced by "sinc_interp_hann" in the next release. The default behavior remains unchanged.
  return ta_resample(audio, orig_sr, new_sr, **params)


# 2. transcription with MMS

In [1]:
import gradio as gr
from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch
import librosa
import json
model_id = "facebook/mms-1b-all"
processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)
speech, sample_rate = librosa.load('enhanced.wav')
if sample_rate != 16000:
        speech = librosa.resample(speech, orig_sr=sample_rate, target_sr=16000)
language_code = 'swh'
processor.tokenizer.set_target_lang(language_code)
model.load_adapter(language_code)
inputs = processor(speech, sampling_rate=16_000, return_tensors="pt")
with torch.no_grad():
        outputs = model(**inputs).logits
ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
transcription

/home/sosa/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-22 11:18:30.193494: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 11:18:30.466059: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 11:18:30.466105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 11:18:30.513211: E ext

: 